In [46]:
import json
import pandas as pd
import requests
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [47]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [48]:
results = sf.query_all(
    """
   select Id, IdentificationId__pc from Account 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Pacientes_SF = df[["Id", "IdentificationId__pc"]]
Pacientes_SF.shape

(1536922, 2)

In [49]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select id,Name,Nombre_Empresa__c,Empresa__c, Empresa_Padre__c from Programas_por_Empresas__c
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_SF = df[["Id", "Nombre_Empresa__c", "Name", "Empresa__c","Empresa_Padre__c"]]
Empresas_SF.head()



,Id,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,a1W8c000007vDN8EAM,Metlife Holding Metlife Laboratorio Maver Ltda.,A-0000,0018c00002axQXrAAM,Metlife
1,a1W8c000007vDN9EAM,Metlife Holding Metlife Aguas Del Valle S.A.,A-0001,0018c00002axQXsAAM,Metlife
2,a1W8c000007vDNAEA2,Metlife Holding Metlife Berliam Spa.,A-0002,0018c00002axQXtAAM,Metlife
3,a1W8c000007vDNBEA2,Metlife Holding Metlife Besalco Maquinarias S.A.,A-0003,0018c00002axQXuAAM,Metlife
4,a1W8c000007vDNCEA2,Metlife Holding Metlife Carlos Escarate Y Cia ...,A-0004,0018c00002axQXvAAM,Metlife


In [50]:
Empresas_SF.shape

(7879, 5)

In [51]:

# Relacion pacientes y empresas
results = sf.query_all(
    """
  select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
    """
)
df = pd.DataFrame(results)
df.shape
Pacientes_empresas = pd.DataFrame(results["records"])
Pacientes_empresas = Pacientes_empresas[
    ["Paciente__c", "Programas_por_Empresas__c"]
]
Pacientes_empresas.head()

,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM


In [52]:
#une Pacientes y el que hace de union Pacientes_empresas
pacientes_y_union = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
pacientes_y_union.head()

,Id,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ
1,0018c00002amLUGAA2,153338949,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083j8uEAA
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI


In [53]:
pacientes_y_union.shape

(1644493, 4)

In [54]:
pacientes_y_empresas_final = pd.merge(
    left=pacientes_y_union,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
pacientes_y_empresas_final.head()

,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ,a1W8c0000083icPEAQ,Metlife Vulco Vulco S.A.,A-1075,0018c00002cXJZeAAO,Metlife
1,0018c00002amLUGAA2,153338949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA,a1W8c0000083j8wEAA,Farmacias Cruz Verde Holding Cruz Verde Farmac...,A-2322,0018c00002cXclVAAS,Farmacias Cruz Verde
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083j8uEAA,a1W8c0000083j8uEAA,Ey Holding Ey Ey,A-2320,0018c00002cXclTAAS,Ey
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI,a1W8c0000083iWhEAI,Minera Abra Holding Minera Abra Minera Abra,A-0721,0018c00002cXJTrAAO,Minera Abra


In [63]:
nuevos = "C:/Users/jmartinez/Documents/cargas/ver.xlsx"
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

,codigo,nombres,apellidos,Unnamed: 3,Unnamed: 4,Unnamed: 5,rut1,Unnamed: 7,rut
0,SA-3124,Carolina Mendoza Abarzua,Carolina Mendoza Abarzua,NaN,Ingreso al programa,2022-12-02,183552252,Clínica de Nutrición,183552252
1,SA-4987,Genesis Antonia Gutierrez Abarca,Genesis Antonia Gutierrez Abarca,NaN,Ingreso al programa,2022-12-21,224488556,Clínica de Nutrición,224488556
2,SA-3905,Fernanda Godoy Miranda,Fernanda Godoy Miranda,NaN,Ingreso,2022-12-12,185384799,Clínica de Deporte,185384799
3,SA-3871,Valeria Elizabe Rojas Parra,Valeria Elizabe Rojas Parra,NaN,Ingreso,2022-12-12,187821444,Clínica de Psicología,187821444
4,SA-3898,Oscar Ignacio Contreras Godoy,Oscar Ignacio Contreras Godoy,NaN,Ingreso,2022-12-12,167466923,Clínica de Nutrición,167466923


In [64]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
#Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])

In [65]:
Pacientes_Archivo.head()

,codigo,nombres,apellidos,Unnamed: 3,Unnamed: 4,Unnamed: 5,rut1,Unnamed: 7,rut,RutEstaoNo
0,SA-3124,Carolina Mendoza Abarzua,Carolina Mendoza Abarzua,NaN,Ingreso al programa,2022-12-02,183552252,Clínica de Nutrición,183552252,True
1,SA-4987,Genesis Antonia Gutierrez Abarca,Genesis Antonia Gutierrez Abarca,NaN,Ingreso al programa,2022-12-21,224488556,Clínica de Nutrición,224488556,True
2,SA-3905,Fernanda Godoy Miranda,Fernanda Godoy Miranda,NaN,Ingreso,2022-12-12,185384799,Clínica de Deporte,185384799,True
3,SA-3871,Valeria Elizabe Rojas Parra,Valeria Elizabe Rojas Parra,NaN,Ingreso,2022-12-12,187821444,Clínica de Psicología,187821444,True
4,SA-3898,Oscar Ignacio Contreras Godoy,Oscar Ignacio Contreras Godoy,NaN,Ingreso,2022-12-12,167466923,Clínica de Nutrición,167466923,True


In [66]:
Pacientes_con_ID = pd.merge(
    left=Pacientes_Archivo,
    right=pacientes_y_empresas_final,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_con_ID.head()


,codigo,nombres,apellidos,Unnamed: 3,Unnamed: 4,Unnamed: 5,rut1,Unnamed: 7,rut,RutEstaoNo,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,SA-3124,Carolina Mendoza Abarzua,Carolina Mendoza Abarzua,NaN,Ingreso al programa,2022-12-02,183552252,Clínica de Nutrición,183552252,True,0018c00002azpJ4AAI,183552252,0018c00002azpJ4AAI,a1W8c000007vDUFEA2,a1W8c000007vDUFEA2,Vida Security Grupo Sqm S.Q.M. Salar S.A.,A-0441,0018c00002axQfXAAU,Vida Security
1,SA-4987,Genesis Antonia Gutierrez Abarca,Genesis Antonia Gutierrez Abarca,NaN,Ingreso al programa,2022-12-21,224488556,Clínica de Nutrición,224488556,True,0018c00002amLbpAAE,224488556,0018c00002amLbpAAE,a1W8c000007vDPvEAM,a1W8c000007vDPvEAM,Metlife Holding Metlife Serv. De Comerc Y Ases...,A-0173,0018c00002axQaeAAE,Metlife
2,SA-3905,Fernanda Godoy Miranda,Fernanda Godoy Miranda,NaN,Ingreso,2022-12-12,185384799,Clínica de Deporte,185384799,True,0018c00002ay6KFAAY,185384799,0018c00002ay6KFAAY,a1W8c000007vDSoEAM,a1W8c000007vDSoEAM,Metlife Holding Metlife Asociacion Chilena De ...,A-0352,0018c00002axQdUAAU,Metlife
3,SA-3871,Valeria Elizabe Rojas Parra,Valeria Elizabe Rojas Parra,NaN,Ingreso,2022-12-12,187821444,Clínica de Psicología,187821444,True,0018c00002bI5vJAAS,187821444,0018c00002bI5vJAAS,a1W8c000007vDT4EAM,a1W8c000007vDT4EAM,Metlife Holding Metlife Provida,A-0368,0018c00002axQdjAAE,Metlife
4,SA-3898,Oscar Ignacio Contreras Godoy,Oscar Ignacio Contreras Godoy,NaN,Ingreso,2022-12-12,167466923,Clínica de Nutrición,167466923,True,0018c00002ao4fCAAQ,167466923,0018c00002ao4fCAAQ,a1W8c000007vDSpEAM,a1W8c000007vDSpEAM,Metlife Clinica Alemana Clinica Alemana,A-0353,0018c00002axQecAAE,Metlife


In [67]:
Pacientes_con_ID.to_excel(
    "C:/Users/jmartinez/Documents/cargas/Pacientes_y_empresas_nuevas.xlsx", index=False
)
